# Generatívne neurónové siete

Na dnešnom cvičení si vysvetlíme princíp fungovania **generatívnych adversiálnych neurónových sietí** (*generative adversarial network* alebo *GAN*). GANká sú pomerne nový typ neurónových sietí, ich architektúra bola popísaná v roku 2017 [1]. Riešia v prvom rade problém generovania nových údajov: teda ak máme k dispozícii trénovacie dáta, dokáže neurónová sieť generovať úplne nové dáta, ktoré sú podobné tým v trénovacej množine? Ako sami uvidíte na dnešnom cvičení, odpoveď je áno.

## 1. Princíp GAN

Hlavný problém pri vývoji neurónových sietí pre generovanie údajov je problém trénovania, pretože na prvý pohľad neexistuje dobrý spôsob na vyhodnotenie presnosti takýchto sietí. Ako by sme dokázali presne vyjadriť, či sa neurónová sieť zlepšuje alebo nie v generovaní (napríklad obrázkov)? Kontrolované učenie by bolo vhodné, keby sme chceli dosiahnuť, aby neurónová sieť reprodukovala obrázky (podobné siete sa nazývajú **autoenkódery** a budeme sa s nimi zaoberať na ďalšom cvičení), ale nemali by sme spôsob na vyhodnotenie nových obrázkov vygenerovaných sieťou. V prípade nekontrolovaného učenia by sme vedeli zistiť, či vygenerovaný obrázok padá do rovnakého zhluku ako obrázky v trénovacej množine, ale jednoduchá architektúra nám nestačí na spresnenie tohto vyhodnotenia. Reinforcement learning ("tretia paradigma" trénovania neurónových sietí, ktorá bude predmetom 11. cvičenia) je tiež nevhodný na tento problém, keďže neexistuje žiadny signál, ktorý by sme vedeli použiť na trénovanie siete a ktorý by vyjadroval, akú presnosť sieť dosiahla.

Riešením problému bola nová architektúra, ktorá sa skladá z dvoch sietí: z generátora a diskriminátora. Tieto dva modely hrajú nekonečnú adversariálnu (teda *zero-sum*) hru: ak je na tom lepšie prvá sieť, znamená to, že druhá sieť prehráva, a bude sa snažiť zlepšiť svoj výkon, a naopak. Všeobecnú architektúru GANiek vidíme na nasledujúcom obrázku:

![GAN architektúra](resources/lab10/gan_architecture.png)

Úlohou generátora je generovať nové obrázky (alebo iný typ údajov) z náhodného vstupu, t.j. šumu. Vyprodukovaný príklad je potom použitý ako vstup diskriminátora. Diskriminátor má jednu úlohu: zistiť, či vstup je reálny (teda z trénovacej množiny) alebo bol vygenerovaný generátorom - je to jednoduchý binárny klasifikátor. Použitím tejto architektúry nemusíme riešiť spôsob trénovania generátora a diskriminátora osobitne, natrénujeme ich spolu.

## 2. Trénovanie GAN

Hlavný problém pri trénovaní generátora osobitne je to, že nemáme k dispozícii očakávané vstupy a výstupy. Vstupný šum vieme vygenerovať náhodne, ale nedokážeme dostať k nemu prislúchajúci očakávaný výstup - z nášho pohľadu vygenerovaný výstup môže byť vhodný pre naše účely alebo len šum podobný vstupu. Teoreticky by sme mohli poprosiť ľudí, aby ohodnotili výstup generátora, či je podľa nich postačujúci alebo nie, ale takýto proces by bol namáhavý a nepraktický. Práve preto pridáme do architektúry diskriminátor, ktorý nahradí takéhoto človeka a naučí sa rozdeliť reálne obrázky od tých generovaných. Oproti človeku má diskriminátor ešte jednu výhodu, t.j. vie konkrétne, ktoré obrázky boli reálne, a ktoré vyprodukované. S takouto architektúrou sú nám známe vstupy aj výstupy:

* vstupom generátora je náhodne vygenerovaný šum, a výstupom je obrázok;
* ak diskriminátor dostane na vstupe obrázok z trénovacej množiny, jeho výstupom má byť 1 (reálny obrázok);
* ak diskriminátor dostane na vstupe obrázok vygenerovaný generátorom, jeho výstupom má byť 0 (fake obrázok).

Práve preto generátor nebudeme trénovať samostatne, ale spojíme ho s diskriminátorom do jednej siete, ktorej vstupom je šum, a očakávaným výstupom je 0 (keďže vždy generujeme fakeový obrázok). Samotný obrázok je skrytý v strede siete ako výstup generátora. Túto architektúru vidíte v dolnej časti obrázka vyššie.

Všeobecne, trénovanie GANiek má nasledujúce kroky:
1. predtrénovanie diskriminátora na reálnych obrázkoch
2. trénovanie kombinovanej siete (generátor + diskriminátor) s náhodným vstupom a očakávaným výstupom 0 na istom počte príkladov
3. trénovanie diskriminátora na rovnakom počte trénovacích príkladov (očakávaný výstup 1)
4. vrátime sa k bodu 2 a pokračujeme v trénovaní až kým nie sme spokojní s výsledkami.

Kým pri klasifikačných úlohách naším cieľom je maximalizovať presnosť klasifikátora (dosiahnuť 100%), pri GANkách presnosť chceme udržiavať ideálne okolo 50% - diskriminátor v takom prípade sa nevie rozhodnúť, či vidí reálny alebo fakeový obrázok, nemá vyššiu presnosť ako náhodný generátor. Keďže sa jedná o binárnu klasifikáciu, nemôžeme byť spokojní ani s vysokou ani s nízkou presnosťou (pri nízkej presnosti by nám stačilo zmeniť výstup klasifikátora a máme lepší klasifikátor).

## 3. Implementácia GANiek

Po teoretickom prehľade sa spustíme do implementácie jednoduchécho GAN modelu, konkrétne použijeme originálnu architektúru s plne prepojenými vrstvami, ktorá zbehne aj na bežných počítačoch pomerne rýchlo. Samozrejme dnes už existuje mnoho variantov GANiek v kombinácii s inými metódami, ich zoznam s kerasovskou implementáciou nájdete [tu](https://github.com/eriklindernoren/Keras-GAN) [2].

Znova použijeme MNIST dataset rukou písaných číslic, hotový kód nájdete [tu](resources/lab10/lab10.py). Ak všetko prebehne bez problémov, našu sieť naučíme písať číslice (porovnajte výstup netrénovanej a natrénovanej siete):

![Výstup netrénovanej siete](resources/lab10/gan_untrained.png)

![Výstup trénovanej siete](resources/lab10/gan_trained.png)

## 4. Problémy pri trénovaní

Aj keď teória za GANkami je jednoduchá, ich implementácia nie je až taká priamočiara. A ako aj pri ďalších jednoduchých systémoch, existuje niekoľko spôsobov ako trénovanie môže zlyhať. Dnes sa pozrieme na dva najčastejšie problémy.

**Diskriminátor dominuje nad generátorom** - v tomto prípade nezáleží na tom, čo vyprodukuje generátor, diskriminátor vždy dokáže dobre rozpoznať fakeové a reálne obrázky. To reálne znamená, že presnosť diskriminátora ostáva okolo 100%, a generátor tak nedostane užitočnú informáciu o svojej funkcionalite: nech urobí čokoľvek, diskriminátor ho stále odhalí, a preto generátor sa nedokáže zlepšiť. Najčastejšou príčinou tohto problému je nevhodná topológia, ktorá je príliš jednoduchá pre riešený problém, alebo predtrénovanie diskriminátora trvá príliš dlho a bol tak pretrénovaný na trénovacích príkladoch.

**Mode collapse** je ďalší častý problém pri trénovaní GANiek a je spôsobený samotným generátorom. Generátor, ako niektorí študenti, je lenivý a bude hľadať najjednoduchší spôsob ako spĺňať svoju úlohu, aj keď dané riešenie nemá žiaden zmysel. To je pre generátor ešte jednoduchšie, keďže nemá žiadne pochopenie toho, čo presne má robiť, preňho existuje len vstupný šum a signál od diskriminátora, ktorý určí, či robí niečo dobre alebo zle. Práve preto sa môže stať, že generátor nájde jeden obrázok, ktorý zmätie diskriminátor, a generátor ho bude vyprodukovať vždy. Alebo identifikuje inú slabinu diskriminátora a bude ju zneužívať na produkovanie obrázkov, ktoré diskriminátor neskôr zle klasifikuje - môže sa to stať najmä ak diskriminátor sa sústredí iba na malú časť vstupu pri rozhodovaní o výsledku klasifikácie. Je ťažké nájsť riešenie pre *mode collapse* keďže nejde o chybnú funkcionalitu ale presne naopak: generátor robí svoju úlohu príliš dobre. Môžete skúsiť upraviť topológiu klasifikátora, aby bol robustnejší, alebo zvýšiť variabilitu náhodne generovaného šumu.

## Doplňujúce úlohy

1. Upravte kód alebo napíšte vlastné riešenie kde zadefinujete vlastný GAN model s použitím konvolúcie (DCGAN). Porovnajte jeho funkcionalitu s jednoduchou GANkom.
2. Súčasné riešenie nášho GANka generuje čísla náhodne. Navrhnite riešenie, ktoré by vygenerovalo požadovanú číslicu. (Váš návrh napíšte do komentárov implementácie úlohy 1.)

## Použité zdroje
1. Goodfellow, Ian, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville, and Yoshua Bengio. "Generative adversarial nets." In Advances in neural information processing systems, pp. 2672-2680. 2014. [available online](http://papers.nips.cc/paper/5423-generative-adversarial-nets)
2. eriklindernoren: Collection of Keras implementations of Generative Adversarial Networks (GANs) suggested in research papers. GitHub [available online](https://github.com/eriklindernoren/Keras-GAN)

## Príprava prostredia

Na budúcom cvičení budeme pracovať s knižnicou [Gym od OpenAI](https://gym.openai.com). Nainštalujete si ju pomocou príkazu:

In [ ]:
python -m pip install gym